<a href="https://colab.research.google.com/github/yma2929/Gen-game/blob/main/Gen_game!.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Balanced_Dataset_with_Challenge_Level.csv to Balanced_Dataset_with_Challenge_Level (1).csv


In [ ]:
#<<Train suggested sports based on player skills model>>
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import joblib

# Load the dataset
df = pd.read_csv("/content/Balanced_Dataset_with_Challenge_Level.csv")

# skills
mlb_sport = MultiLabelBinarizer()
skills_encoded = mlb_sport.fit_transform(df["Skills"])
skills_df = pd.DataFrame(skills_encoded, columns=mlb_sport.classes_)

# Features: skills + height + weight
X = pd.concat([skills_df.reset_index(drop=True), df[["Height_cm", "Weight_kg"]].reset_index(drop=True)], axis=1)
y = df["Preferred_Sport"]

# Split & train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
sport_model = RandomForestClassifier(random_state=42)
sport_model.fit(X_train, y_train)

y_pred = sport_model.predict(X_test)
print("=== Sport Classification Report ===")
print(classification_report(y_test, y_pred))

# Save model and encoder
joblib.dump(sport_model, "trained_sport_model.pkl")
joblib.dump(mlb_sport, "skills_encoder.pkl")

=== Sport Classification Report ===
               precision    recall  f1-score   support

   Basketball       1.00      1.00      1.00         4
       Boxing       0.71      0.83      0.77         6
        Chess       1.00      1.00      1.00         3
      Esports       1.00      0.67      0.80         6
   Gymnastics       0.78      0.88      0.82         8
      Parkour       0.62      0.83      0.71         6
       Soccer       0.83      0.71      0.77         7
     Swimming       0.88      0.88      0.88         8
Track & Field       1.00      0.83      0.91         6

     accuracy                           0.83        54
    macro avg       0.87      0.85      0.85        54
 weighted avg       0.85      0.83      0.84        54



['skills_encoder.pkl']

In [ ]:
#<<predict suggested sports based on player skills>>
import numpy as np
import joblib

# Load model and encoder
sport_model = joblib.load("trained_sport_model.pkl")
mlb_sport = joblib.load("skills_encoder.pkl")

# Input from user
skills_input = input("Enter player's skills (comma-separated): ")
height = int(input("Enter player's height (cm): "))
weight = int(input("Enter player's weight (kg): "))

# Process skills
skills_list = [s.strip().capitalize() for s in skills_input.split(",")]
skills_encoded = mlb_sport.transform([skills_list])
input_vector = np.concatenate((skills_encoded[0], [height, weight])).reshape(1, -1)

# Predict
predicted_sport = sport_model.predict(input_vector)
print("\n suggested sports based on player skills:", predicted_sport[0])

Enter player's skills (comma-separated): Speed , Power  , Focus
Enter player's height (cm): 178
Enter player's weight (kg): 78

 suggested sports based on player skills: Basketball


/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:909: UserWarning: unknown class(es) ['Focus', 'Power', 'Speed'] will be ignored
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [ ]:
#<<Train challenge Level Model>>

# Skills already processed
mlb_challenge = MultiLabelBinarizer()
skills_encoded_challenge = mlb_challenge.fit_transform(df["Skills"])
skills_df_challenge = pd.DataFrame(skills_encoded_challenge, columns=mlb_challenge.classes_)

# Features for challenge model
X_c = pd.concat([skills_df_challenge.reset_index(drop=True), df[["Height_cm", "Weight_kg"]].reset_index(drop=True)], axis=1)
y_c = df["Challenge_Level"]

# Train-test split
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X_c, y_c, test_size=0.2, random_state=42)
challenge_model = RandomForestClassifier(random_state=42)
challenge_model.fit(X_train_c, y_train_c)

y_pred_c = challenge_model.predict(X_test_c)
print("=== Challenge Level Classification Report ===")
print(classification_report(y_test_c, y_pred_c))

# Save model and encoder
joblib.dump(challenge_model, "trained_challenge_model.pkl")
joblib.dump(mlb_challenge, "challenge_skills_encoder.pkl")

=== Challenge Level Classification Report ===
              precision    recall  f1-score   support

        Easy       0.83      0.80      0.81        30
        Hard       0.50      0.58      0.54        12
      Medium       0.55      0.50      0.52        12

    accuracy                           0.69        54
   macro avg       0.62      0.63      0.62        54
weighted avg       0.69      0.69      0.69        54



['challenge_skills_encoder.pkl']

In [ ]:
#<< Predict challenge Level>>
import numpy as np
import joblib

# Load challenge model and encoder
challenge_model = joblib.load("trained_challenge_model.pkl")
mlb_challenge = joblib.load("challenge_skills_encoder.pkl")

skills_input = input("Enter player's skills (comma-separated): ")
height = int(input("Enter player's height (cm): "))
weight = int(input("Enter player's weight (kg): "))

skills_list = [s.strip().capitalize() for s in skills_input.split(",")]
skills_encoded = mlb_challenge.transform([skills_list])
input_vector = np.concatenate((skills_encoded[0], [height, weight])).reshape(1, -1)

predicted_level = challenge_model.predict(input_vector)
print("\nRecommended Challenge Level:", predicted_level[0])

Enter player's skills (comma-separated): Teamwork , Flexibility
Enter player's height (cm): 179
Enter player's weight (kg): 68

Recommended Challenge Level: Easy


/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:909: UserWarning: unknown class(es) ['Flexibility', 'Teamwork'] will be ignored
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
